In [1]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)


In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)


In [3]:
# Reversing the text
for item in range(len(train_data)):
  train_data[item].text[::] = train_data[item].text[::-1]

In [4]:
print(train_data[1].text[::])

['.', 'versions', 'video', 'home', 'language', '-', 'English', 'no', 'are', 'there', 'though', 'even', 'out', 'seeking', 'worth', 'Well', '.', 'along', 'goes', 'it', 'as', 'nicely', 'itself', 'explains', 'story', 'the', ';', 'be', 'to', 'need', "n't", 'wo', 'legend', 'the', 'with', 'unfamiliar', 'Those', '.', 'movie', 'looking', '-', 'terrific', 'and', 'spellbinding', ',', 'gory', ',', 'dark', 'a', 'produce', 'to', 'head', 'its', 'on', 'it', 'stood', 'and', 'Benkei', 'Musashibo', 'of', 'myth', 'old', 'the', 'taken', 'has', 'Ishii', 'Sogo']


In [7]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [8]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [9]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [10]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        num_lstm_layers = 3

        for i in range(num_lstm_layers):

          self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [11]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [13]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [14]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.0665, -0.0531,  2.1597,  ..., -0.2276,  2.1432, -0.6743],
        [-1.9025,  1.1764,  0.3719,  ..., -0.9814,  0.4348,  0.9766],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2277, -0.4441, -0.4130,  ...,  0.8015, -1.0104,  1.5693],
        [-0.7090, -0.6751,  1.7083,  ..., -1.2000, -0.6648, -0.8007],
        [-0.6064, -0.0718, -0.5077,  ...,  0.0370,  1.1486,  0.3894]])

In [15]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2277, -0.4441, -0.4130,  ...,  0.8015, -1.0104,  1.5693],
        [-0.7090, -0.6751,  1.7083,  ..., -1.2000, -0.6648, -0.8007],
        [-0.6064, -0.0718, -0.5077,  ...,  0.0370,  1.1486,  0.3894]])


In [16]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [17]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [18]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [27]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        #text_lengths = text_lengths.cpu()
        text, text_lengths = batch.text
        text_lengths = text_lengths.cpu()
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [31]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            text_lengths = text_lengths.cpu()
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [38]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 41s
	Train Loss: 0.175 | Train Acc: 93.50%
	 Val. Loss: 0.269 |  Val. Acc: 89.46%
Epoch: 02 | Epoch Time: 0m 41s
	Train Loss: 0.134 | Train Acc: 95.29%
	 Val. Loss: 0.287 |  Val. Acc: 89.12%
Epoch: 03 | Epoch Time: 0m 41s
	Train Loss: 0.099 | Train Acc: 96.66%
	 Val. Loss: 0.304 |  Val. Acc: 88.87%
Epoch: 04 | Epoch Time: 0m 41s
	Train Loss: 0.077 | Train Acc: 97.44%
	 Val. Loss: 0.330 |  Val. Acc: 89.12%
Epoch: 05 | Epoch Time: 0m 41s
	Train Loss: 0.052 | Train Acc: 98.24%
	 Val. Loss: 0.390 |  Val. Acc: 88.75%
Epoch: 06 | Epoch Time: 0m 41s
	Train Loss: 0.040 | Train Acc: 98.81%
	 Val. Loss: 0.443 |  Val. Acc: 89.28%
Epoch: 07 | Epoch Time: 0m 41s
	Train Loss: 0.035 | Train Acc: 98.84%
	 Val. Loss: 0.422 |  Val. Acc: 89.15%
Epoch: 08 | Epoch Time: 0m 41s
	Train Loss: 0.025 | Train Acc: 99.28%
	 Val. Loss: 0.488 |  Val. Acc: 88.65%
Epoch: 09 | Epoch Time: 0m 41s
	Train Loss: 0.022 | Train Acc: 99.36%
	 Val. Loss: 0.532 |  Val. Acc: 89.26%
Epoch: 10 | Epoch T

In [39]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 0.368 | Test Acc: 85.33%


In [40]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()


In [41]:
predict_sentiment(model, "This film is terrible")

0.008010591380298138

In [42]:
predict_sentiment(model, "This film is great")

0.877715528011322